In [5]:
import pandas as pd
import numpy as np

In [11]:
featuresData = pd.read_csv("../dataset/kopo_decision_tree_all_new.csv")
len(featuresData.ITEM.unique())
featuresData.head(1)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,A01,PG01,P01,ITEM001,201538,2015,38,1,N,4,N,0.0


In [12]:
groupKey = ["REGIONID","PRODUCTGROUP","PRODUCT","ITEM"]

In [23]:
groupData = featuresData.groupby(groupKey)["YEARWEEK"].agg(["size"]).reset_index()
groupData.head(2)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,size
0,A01,PG01,P01,ITEM001,3
1,A01,PG01,P01,ITEM002,2


In [24]:
groupData.rename(columns={"size":"KNOB"}, inplace=True)
groupData.head(1)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,KNOB
0,A01,PG01,P01,ITEM001,3


In [29]:
mergedData = pd.merge(left=featuresData, right=groupData, on= groupKey, how = "left")

In [38]:
maxKnob = mergedData.KNOB.max()

## 1. 지역/ 상품/ 아이템별 실적이 최대인 데이터만 필터링

In [41]:
cleansedData = mergedData[mergedData.KNOB >= maxKnob]

## 2. 그룹단위 머신러닝 수행

In [106]:
from sklearn import tree
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [97]:
def predict_model(onegroup):

#     groupResult = cleansedData.groupby(groupKey) # 그룹단위 예측 수행

#     list(groupResult.groups)[0]
#     len(groupResult)

#     onegroup = groupResult.get_group(list(groupResult.groups)[0])  # 0 ~ 51 한개의 그룹만 가지고 오기
#     onegroup.head(1)

    eachgroup = onegroup.reset_index(drop=True)

#     eachgroup.shape

    # * 1. 특성선정 및 데이터 분리

    corrdf = eachgroup.corr()
    yearweekStd = 201701

    features = list (corrdf[ (abs(corrdf.QTY) > 0.5) & (abs(corrdf.QTY) < 1)].index)
    
    if (len(features) > 0):

        label = ["QTY"]

        ###1. feature 개수를 확인한다. 
        ###2. feature 개수가 0인 경우 해당 그룹의 예측 값음 0으로 세팅한다. 
        ###3. feature 개수가 1개 이상인 경우 예측값음 머신러닝으로 수행
        ###plus 
        ###1.예측 모델은 3개 이상 구현
        ###2.그룹별 mae 값 생성
        ###3.베스트 모델 추천

        

        trainingData_features = eachgroup[eachgroup.YEARWEEK < yearweekStd][features]
        trainingData_label = eachgroup[eachgroup.YEARWEEK < yearweekStd][label]
        testData_features = eachgroup[eachgroup.YEARWEEK >= yearweekStd][features]
        testData_label = eachgroup[eachgroup.YEARWEEK >= yearweekStd][label]
        testData_all = eachgroup[eachgroup.YEARWEEK >= yearweekStd]


    #     print(trainingData_features.shape)
    #     print(trainingData_label.shape)
    #     print(testData_features.shape)
    #     print(testData_label.shape)

        # * 2. 모델선언
        model_method = tree.DecisionTreeRegressor(random_state=1)

        # * 3. 학습
        model = model_method.fit(trainingData_features,trainingData_label)

        # * 4. 예측
        predictValue = model.predict(testData_features)
        predictValue
        testData_all["DT_PREDICT"] = predictValue
    else:
        testData_all = eachgroup[eachgroup.YEARWEEK >= yearweekStd]
        testData_all["DT_PREDICT"] = 0
    
    return testData_all

In [102]:
def predict_model(onegroup):

#     groupResult = cleansedData.groupby(groupKey) # 그룹단위 예측 수행

#     list(groupResult.groups)[0]
#     len(groupResult)

#     onegroup = groupResult.get_group(list(groupResult.groups)[0])  # 0 ~ 51 한개의 그룹만 가지고 오기
#     onegroup.head(1)

    eachgroup = onegroup.reset_index(drop=True)

#     eachgroup.shape

    # * 1. 특성선정 및 데이터 분리

    corrdf = eachgroup.corr()
    yearweekStd = 201701

    features = list (corrdf[ (abs(corrdf.QTY) > 0.5) & (abs(corrdf.QTY) < 1)].index)
    
    if (len(features) > 0):

        label = ["QTY"]

        ###1. feature 개수를 확인한다. 
        ###2. feature 개수가 0인 경우 해당 그룹의 예측 값음 0으로 세팅한다. 
        ###3. feature 개수가 1개 이상인 경우 예측값음 머신러닝으로 수행
        ###plus 
        ###1.예측 모델은 3개 이상 구현
        ###2.그룹별 mae 값 생성
        ###3.베스트 모델 추천

        

        trainingData_features = eachgroup[eachgroup.YEARWEEK < yearweekStd][features]
        trainingData_label = eachgroup[eachgroup.YEARWEEK < yearweekStd][label]
        testData_features = eachgroup[eachgroup.YEARWEEK >= yearweekStd][features]
        testData_label = eachgroup[eachgroup.YEARWEEK >= yearweekStd][label]
        testData_all = eachgroup[eachgroup.YEARWEEK >= yearweekStd]


    #     print(trainingData_features.shape)
    #     print(trainingData_label.shape)
    #     print(testData_features.shape)
    #     print(testData_label.shape)

        # * 2. 모델선언
        model_method_dt = tree.DecisionTreeRegressor(random_state =1)
        model_method_lr = linear_model.LinearRegression()
        model_method_rf = ensemble.RandomForestRegressor(random_state =1, n_estimators= 10)

        # * 3. 학습
        model_dt =model_method_dt.fit (trainingData_features ,trainingData_label)
        model_lr =model_method_lr.fit (trainingData_features ,trainingData_label)
        model_rf =model_method_rf.fit (trainingData_features ,trainingData_label)

        # * 4. 예측
        predict_dt = model_dt.predict(testData_features)
        predict_lr = model_lr.predict(testData_features)
        predict_rf = model_rf.predict(testData_features)
        testData_all["DT_PREDICT"] = predict_dt
        testData_all["LR_PREDICT"] = predict_lr
        testData_all["RF_PREDICT"] = predict_rf
    else:
        testData_all = eachgroup[eachgroup.YEARWEEK >= yearweekStd]
        testData_all["DT_PREDICT"] = 0
        testData_all["LR_PREDICT"] = 0
        testData_all["RF_PREDICT"] = 0
    
    return testData_all

In [103]:
finalResult = cleansedData.groupby(groupKey).apply(predict_model)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y

In [110]:
finalResult.head(2)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,KNOB,DT_PREDICT
105,A01,PG02,P03,ITEM043,201701,2017,1,470,Y,1,Y,0.240964,146,0
106,A01,PG02,P03,ITEM043,201702,2017,2,293,N,4,Y,0.240964,146,0


In [137]:
mae_dt = mean_absolute_error(finalResult["QTY"], finalResult.DT_PREDICT)
mae_rf = mean_absolute_error(finalResult["QTY"], finalResult.LR_PREDICT)
mae_lr = mean_absolute_error(finalResult["QTY"], finalResult.RF_PREDICT)
dicMae = [["Decision Trees" , mae_dt],[ "Linear Regression" , mae_rf], ["Random Forests" , mae_lr]]
dicMae = sorted(dicMae, key=lambda t :t[1])
print("추천 알고리즘 : " + str(dicMae[0][0]))

추천 알고리즘 : Random Forests


In [112]:
pd.DataFrame([[mae_dt,mae_rf,mae_lr]], columns=["mea_dt","mae_rf","mae_lr"])

,mea_dt,mae_rf,mae_lr
0,79.981315,88.251373,79.281396


# dicMae = sorted(dicMae, key=lambda t :t[1])
dicMae

In [ ]:
{'mae_dt': 79.98131466694332,
 'mae_rf': 88.25137254883843,
 'mae_lr': 79.2813964458112}

In [56]:
eachgroup.YEARWEEK.describe()

count       146.000000
mean     201616.917808
std          79.102858
min      201501.000000
25%      201537.250000
50%      201620.500000
75%      201704.750000
max      201741.000000
Name: YEARWEEK, dtype: float64